In [4]:
import pandas as pd
from collections import defaultdict

# Carica i dati
anagrafica = pd.read_excel("output_anagrafica.xlsx")
spareorder = pd.read_excel("output_spareorder.xlsx")

# Prepara struttura dati
json_out = {}

# Ciclo su tutti i modelli presenti in spareorder
for modello in spareorder['modello'].unique():
    modello_dict = {}
    
    # 1. Spareorders: tutte le date con valore per questo modello
    df_orders = spareorder[spareorder['modello'] == modello]
    modello_dict['spareorders'] = [
        {'data': row['data'], 'valore': row['valore']} for _, row in df_orders.iterrows()
    ]
    
    # 2. Componenti: dalla anagrafica, nome componente (descrizione) e quantità
    df_ana = anagrafica[anagrafica['modello'] == modello]
    componenti = {row['codice']: int(row['quantità']) for _, row in df_ana.iterrows()}
    modello_dict['componenti'] = componenti
    
    # 3. Stat: raggruppa per stat e somma quantità
    stat_dict = df_ana.groupby('stat')['quantità'].sum().to_dict()
    # Conversione a int (evita tipi numpy)
    stat_dict = {k: int(v) for k, v in stat_dict.items()}
    modello_dict['stat'] = stat_dict
    
    json_out[modello] = modello_dict

# Se vuoi salvare il JSON:
import json
with open("output_modelli.json", "w", encoding="utf8") as f:
    json.dump(json_out, f, indent=2, ensure_ascii=False)


In [5]:
for modello, dati in json_out.items():
    num_componenti = len(dati.get('componenti', {}))
    num_stat = len(dati.get('stat', {}))
    print(f"Modello: {modello}")
    print(f"  Numero componenti: {num_componenti}")
    print(f"  Numero stat: {num_stat}")
    print("-" * 30)

Modello: KM720VC
  Numero componenti: 21
  Numero stat: 20
------------------------------
Modello: SM720CWW
  Numero componenti: 23
  Numero stat: 21
------------------------------
Modello: SM720CY6
  Numero componenti: 22
  Numero stat: 20
------------------------------
Modello: WM720MG
  Numero componenti: 22
  Numero stat: 20
------------------------------
Modello: MWHN-20WAG
  Numero componenti: 24
  Numero stat: 21
------------------------------
Modello: SM207S
  Numero componenti: 20
  Numero stat: 18
------------------------------
Modello: MWHN-20WA
  Numero componenti: 22
  Numero stat: 18
------------------------------
Modello: MG2070G
  Numero componenti: 22
  Numero stat: 20
------------------------------
Modello: VMW720G
  Numero componenti: 23
  Numero stat: 21
------------------------------
Modello: MWHN20W
  Numero componenti: 20
  Numero stat: 19
------------------------------
Modello: MWHN20B
  Numero componenti: 20
  Numero stat: 19
------------------------------
Mode

Chiaro, il risultato finale sarà un JSON in cui per ogni modello abbiamo un array di oggetti, uno per ogni data di ordine, con:

la data

per ciascuna componente: il totale ordinato (quantità componente × valore della data)

per ciascun stat: il totale ordinato (quantità stat × valore della data)

In [6]:
# Carica il JSON dei modelli
with open("output_modelli.json", "r", encoding="utf8") as f:
    json_out = json.load(f)

# Output finale
risultato = {}

for modello, dati in json_out.items():
    output_modello = []
    # Lista di ordini per modello
    for ordine in dati['spareorders']:
        data = ordine['data']
        valore = ordine['valore']
        
        # Calcola i totali per componenti
        comp_totali = {comp: qta * valore for comp, qta in dati.get('componenti', {}).items()}
        
        # Calcola i totali per stat
        stat_totali = {stat: qta * valore for stat, qta in dati.get('stat', {}).items()}
        
        output_modello.append({
            "data": data,
            "componenti": comp_totali,
            "stat": stat_totali
        })
    risultato[modello] = output_modello

# Salva il nuovo json
with open("output_modelli_per_data.json", "w", encoding="utf8") as f:
    json.dump(risultato, f, indent=2, ensure_ascii=False)
